In [1]:
# Install required packages.
#import os
#import torch
#os.environ['TORCH'] = torch.__version__
#print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class MySpMM(torch.autograd.Function):
	@staticmethod
	def forward(ctx, sp_mat, dense_mat):
		ctx.save_for_backward(sp_mat, dense_mat)
		return torch.mm(sp_mat, dense_mat)

	@staticmethod
	def backward(ctx, grad_output):
		sp_mat, dense_mat = ctx.saved_variables
		grad_matrix1 = grad_matrix2 = None

		assert not ctx.needs_input_grad[0]
		if ctx.needs_input_grad[1]:
			grad_matrix2 = Variable(torch.mm(sp_mat.data.t(), grad_output.data))

		return grad_matrix1, grad_matrix2


def gnn_spmm(sp_mat, dense_mat):
	return MySpMM.apply(sp_mat, dense_mat)